In [121]:
import numpy as np
import re
import matplotlib.pyplot as plt


In [122]:
def plotVertexList(list, polyList): 
    x = []
    y = []
    polyX = []
    polyY = []
    
    for index, i in enumerate(list): 
        if index % 2 == 0: 
            x.append(i)
        else: 
            y.append(i* -1) 
    for index, i in enumerate(polyList): 
        if index % 2 == 0: 
            polyX.append(i)
        else:
            polyY.append(i * -1)
    #print(x, '\n',y)
    %matplotlib widget
    plt.scatter(x,y, alpha=0.3)
    plt.plot(x,y, alpha=0.2)
    plt.plot(polyX, polyY, alpha=0.3)
    plt.show()
    
    

In [123]:
d_base = "M78.3,21.3c-8.6-0.1-16.7,4.4-21,11.9c-4.3-7.5-12.4-12-21-11.9c-11.7,0-23.1,8-23.1,22.7c0,17.1,20.5,34.6,44.1,58.1c23.6-23.4,44.1-40.9,44.1-58.1C101.3,29.3,89.9,21.3,78.3,21.3"
#d_base = "M0.0,0.0c0,6,15.7,13.1,32.3,7C1.0,3.0, 0.0,0.0, 0.0,0.0"
#d_base = "M0.0,0.0c0.0 -0.5, 2.0-0.5, 2.0,0.0"
#d_base = "M89.6,3c22.3,31-42.5,66.7-42.5,66.7S3.8,40.7,3,11.6"
#d_base = "M47.1,84.5c0,0-43.3-29-44.1-58.1s37.3-29,44.1-10.8"

d_base = "M47.1,89.5c0,0-43.3-29-44.1-58.1s37.3-29,44.1-10.8c0,0,6.5-25,27.5-15.4C141.1,35.4,47.1,89.5,47.1,89.5z"

d_controls = re.split("([,cCMSs-z])", d_base)
D = []

prefix = ''
for d in d_controls: 
    if d != '' and d != ',' and d != :  #remove character 
        if d == '-': #prefix
            prefix = '-'
        else: 
            D.append(prefix + d)
            prefix = ''

D_grouped = []

i = 0; 
while (i+3 <= len(D)): 
    if D[i] == 'c' or D[i] == 'C':
        D_grouped.append([D[i], D[i+1], D[i+2], D[i+3], D[i+4], D[i+5], D[i+6]])
        i += 7
    elif D[i] == 'M':
        D_grouped.append([D[i], D[i+1], D[i+2]])
        i += 3
    elif D[i] == 'S' or D[i] == 's': 
        D_grouped.append([D[i], D[i+1], D[i+2], D[i+3], D[i+4]])
        i += 5
    else: 
        i += 1
        D_grouped.append(['ERROR'])
    
print("D: ", D)
print("\nD_grouped", D_grouped)
    

D:  ['M', '47.1', '89.5', 'c', '0', '0', '-43.3', '-29', '-44.1', '-58.1', 's', '37.3', '-29', '44.1', '-10.8', 'c', '0', '0', '6.5', '-25', '27.5', '-15.4', 'C', '141.1', '35.4', '47.1', '89.5', '47.1', '89.5z']

D_grouped [['M', '47.1', '89.5'], ['c', '0', '0', '-43.3', '-29', '-44.1', '-58.1'], ['s', '37.3', '-29', '44.1', '-10.8'], ['c', '0', '0', '6.5', '-25', '27.5', '-15.4'], ['C', '141.1', '35.4', '47.1', '89.5', '47.1', '89.5z']]


In [124]:

#### create the vertex list 
vertex = []
polyVert = []
mode = "start"
lastX = 0.0
lastY = 0.0
isX = True
for d in D_grouped: 
    if d[0] == 'M':  # first Point
        lastX = float(d[1])
        lastY = float(d[2])
        #vertex.append(float(lastX))
        #vertex.append(float(lastY))
    if d[0] == 'C': # absolute Point
        vertex.append(lastX)
        vertex.append(lastY)
        vertex.append(float(d[1]))
        vertex.append(float(d[2]))
        vertex.append(float(d[3]))
        vertex.append(float(d[4]))
        vertex.append(float(d[5]))
        vertex.append(float(d[6]))
        lastX = float(d[5])
        lastY = float(d[6])
        print("Absoulte Point")
    if d[0] == 'c':  # relative Points
        vertex.append(lastX)
        vertex.append(lastY)
        vertex.append(float(d[1]) + lastX)
        vertex.append(float(d[2]) + lastY)
        vertex.append(float(d[3])  + lastX)
        vertex.append(float(d[4])  + lastY)
        vertex.append(float(d[5]) + lastX)
        vertex.append(float(d[6]) + lastY)
        lastX = float(d[5]) + lastX
        lastY = float(d[6]) + lastY
    if d[0] == 'S': 
        #calc c1
        d_x = vertex[len(vertex) - 4] - vertex[len(vertex) - 2]
        d_y = vertex[len(vertex) - 3] - vertex[len(vertex) - 1]
        
        vertex.append(lastX)
        vertex.append(lastY)
        vertex.append(lastX - d_x) #reflection of prev. control *-1
        vertex.append(lastY - d_y)
        vertex.append(float(d[1]))
        vertex.append(float(d[2]))
        vertex.append(float(d[3]))
        vertex.append(float(d[4]))
        lastX = float(d[3])
        lastY = float(d[4])
    if d[0] == "s": 
        #calc c1
        d_x = vertex[len(vertex) - 4] - vertex[len(vertex) - 2]
        d_y = vertex[len(vertex) - 3] - vertex[len(vertex) - 1]

        vertex.append(lastX)
        vertex.append(lastY)
        vertex.append(lastX - d_x) #reflection of prev. control *-1
        vertex.append(lastY - d_y)
        vertex.append(float(d[1]) + lastX )
        vertex.append(float(d[2]) + lastY )
        vertex.append(float(d[3]) + lastX )
        vertex.append(float(d[4]) + lastY)
        lastX = vertex[len(vertex) - 2]
        lastY = vertex[len(vertex) - 1]
        
        
            

print('\nvertex:', vertex)

i = 0
interval = 1
while(i < len(vertex) ): 
    points = np.array([[vertex[i], vertex[i+1], 0, 1], [vertex[i+2], vertex[i+3], 0 ,1], [vertex[i+4], vertex[i+5], 0, 1], [vertex[i+6], vertex[i+7], 0, 1]])
    m = np.array([[-1, 3, -3, 1], [3,-6,3,0],[-3, 3, 0,0],[1, 0,0,0]])
    mp = np.matmul(m, points) #points * m
    delta = points[3] - points[0]
    magDelta = np.linalg.norm(delta)
    intNorm = interval / (magDelta)
    
    u = 0.0
    while(u <= 1.0):
        uvec = np.array([u*u*u, u*u, u, 1.0])
        pu4 = np.matmul( uvec, mp) #mp * uvec
        polyVert.append(pu4[0])
        polyVert.append(pu4[1])
        u += intNorm
    i += 8
    
plotVertexList(vertex, polyVert)

ValueError: could not convert string to float: '89.5z'

In [ ]:
<!-- Generator: Adobe Illustrator 25.2.1, SVG Export Plug-In  -->
<svg version="1.1" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" x="0px" y="0px" width="101.4px"
	 height="92.5px" viewBox="0 0 101.4 92.5" style="overflow:visible;enable-background:new 0 0 101.4 92.5;" xml:space="preserve">
<style type="text/css">
	.st0{fill:none;stroke:#FFCC00;stroke-width:6;stroke-linecap:round;stroke-linejoin:round;}
</style>
<defs>
</defs>
<path class="st0" d="M47.1,89.5c0,0-43.3-29-44.1-58.1s37.3-29,44.1-10.8c0,0,6.5-25,27.5-15.4C141.1,35.4,47.1,89.5,47.1,89.5z"/>
</svg>
